In [1]:
import os, sys
sys.path.append('/data/spark15/python/')

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (18, 9)
import seaborn as sb
import numpy as np
import pandas as pd

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
from pyspark.sql.types import *
import pyspark.sql.functions

sc = SparkContext("local", "hospital_compare")
sqlContext = SQLContext(sc)
hc = HiveContext(sc)

### What states are models of high-quality care?

In [3]:
print(hc.tableNames())

[u'effective_care', u'effective_care_scores', u'hospitals', u'hospitals_reduced', u'measures', u'readmissions', u'readmissions_reduced', u'survey_responses', u'survey_responses_reduced']


In [4]:
hospitals = hc.sql('select * from hospitals_reduced').toPandas()
effective_care = hc.sql('select * from effective_care_scores').toPandas()

In [13]:
hospitals['hospital_overall_rating'] = hospitals['hospital_overall_rating'].replace('Not Available', np.nan).apply(pd.to_numeric)

In [20]:
hospitals.groupby('state').mean().sort_values(by='hospital_overall_rating', ascending=False).head(10)

,hospital_overall_rating
state,
SD,3.857143
DE,3.666667
WI,3.611111
MN,3.531646
ID,3.526316
IN,3.481481
NH,3.480000
CO,3.450980
UT,3.448276


In [23]:
df = pd.merge(hospitals, effective_care, how='inner', left_on='provider_id', right_on='provider_id')

In [34]:
results = df.set_index(['state', 'provider_id', 'hospital_name', 'measure_id'])[['score']].unstack()
results = results.apply(pd.to_numeric)
results.loc[:, 'Average_Score'] = results.mean(axis=1)
results.loc[:, 'StDev_Score'] = results.std(axis=1)

In [41]:
results.mean(level='state', axis=0).sort_values(by='Average_Score', ascending=False).head(10)

score                                                      \
measure_id ED_1b ED_2b IMM_2 IMM_3_OP_27_FAC_ADHPCT OP_1 OP_18b OP_2   
state                                                                  
VI           NaN   NaN   NaN                    NaN  NaN    NaN  NaN   
WV           NaN   NaN   NaN                    NaN  NaN    NaN  NaN   
AK           NaN   NaN   NaN                    NaN  NaN    NaN  NaN   
WY           NaN   5.0   NaN                    NaN  NaN    NaN  NaN   
NC           NaN   0.0   NaN                    NaN  NaN    NaN  NaN   
KY           NaN   NaN   NaN                    NaN  NaN    NaN  NaN   
ME           NaN   NaN   NaN                    NaN  NaN    NaN  NaN   
OK           NaN   NaN   NaN                    NaN  NaN    NaN  NaN   
VT           NaN   NaN   NaN                    NaN  NaN    NaN  NaN   
WI           NaN   2.5   NaN                    NaN  NaN    NaN  NaN   

                                         ...                                 \
measure_id     OP_20 OP_21     OP_22     ...     OP_31 OP_3b OP_4      OP_5   
state                                    ...                                  
VI               NaN   NaN       NaN     ...       NaN   NaN  NaN       NaN   
WV          8.000000   NaN  1.945946     ...       NaN   NaN  NaN  6.695652   
AK          8.500000   NaN  1.000000     ...       NaN   NaN  NaN  7.600000   
WY          8.333333   NaN  0.692308     ...       NaN   NaN  NaN  5.625000   
NC          9.000000   NaN  2.448276     ...       NaN   NaN  NaN  5.952381   
KY          6.000000   NaN  1.917808     ...       0.0   NaN  NaN  5.568627   
ME          7.500000   NaN  1.280000     ...       NaN   NaN  NaN  6.000000   
OK          6.588235   NaN  1.817073     ...       NaN   NaN  NaN  5.782609   
VT          4.000000   NaN  1.166667     ...       NaN   NaN  NaN  6.000000   
WI          7.000000   NaN  0.633333     ...       NaN   NaN  NaN  6.106061   

                                           Average_Score StDev_Score  
measure_id     PC_01 STK_4 VTE_5     VTE_6                            
state                                                                 
VI          6.000000   NaN   NaN       NaN      6.000000    0.000000  
WV          2.333333   NaN   NaN  1.444444      3.371795    1.658770  
AK          1.428571   NaN   NaN  4.000000      3.333333    2.180966  
WY          2.333333   NaN   NaN  8.000000      3.217544    1.159381  
NC          1.438356   NaN   NaN  1.562500      3.068841    2.053104  
KY          1.418605   NaN   NaN  2.000000      2.990506    1.473358  
ME          1.684211   NaN   NaN  0.500000      2.974138    1.696850  
OK          1.437500   0.0   NaN  1.625000      2.944624    1.507994  
VT          2.500000   NaN   NaN  8.000000      2.904762    1.520182  
WI          1.597222   0.0   NaN  1.565217      2.891141    1.986965  

[10 rows x 23 columns]